Assignment 3

# Langchain Agent and Tools: 

* With Bing Search 

* Answering questions from titanic.CSV file.  

    For example: how many passengers have more than 2 siblings? 


#### Configure OpenAI Settings

In [1]:
import os
import openai
from dotenv import load_dotenv
# Set up Azure OpenAI
load_dotenv()

openai.api_type = "azure"

AZURE_OPENAI_API_VERSION = os.getenv("AAG_AZURE_OPENAI_API_VERSION")
openai.api_version = AZURE_OPENAI_API_VERSION

AZURE_OPENAI_API_KEY = os.getenv("AAG_AZURE_OPENAI_API_KEY").strip()
assert AZURE_OPENAI_API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = AZURE_OPENAI_API_KEY

AZURE_OPENAI_ENDPOINT = os.getenv("AAG_AZURE_OPENAI_ENDPOINT","").strip()
assert AZURE_OPENAI_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
openai.api_base = AZURE_OPENAI_ENDPOINT

# Deployment for Chat
# DEPLOYMENT_NAME_CHAT = os.getenv('DEPLOYMENT_NAME_CHAT')
DEPLOYMENT_NAME_CHAT = os.getenv('AAG_DEPLOYMENT_NAME_CHAT_16K')

# Deployment for embedding
DEPLOYMENT_NAME_EMBEDDING = os.getenv("AAG_DEPLOYMENT_NAME_EMBEDDING")
model: str = DEPLOYMENT_NAME_EMBEDDING

# Deployment for embedding
BING_SUBSCRIPTION_KEY = os.getenv("BING_SUBSCRIPTION_KEY")

#### Summarization - langchain: load_summarize_chain

In [18]:
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain

from langchain.chat_models import AzureChatOpenAI

# Construct Azure OpenAI
llm = AzureChatOpenAI(
    openai_api_type = "azure",
    api_version = AZURE_OPENAI_API_VERSION,
    azure_endpoint = AZURE_OPENAI_ENDPOINT,    
    api_key = AZURE_OPENAI_API_KEY,
    azure_deployment = DEPLOYMENT_NAME_CHAT,
    temperature=0)

def generate_response(txt):
    # Split text
    text_splitter = CharacterTextSplitter()
    texts = text_splitter.split_text(txt)
    # Create multiple documents
    docs = [Document(page_content=t) for t in texts]
    # Text summarization
    chain = load_summarize_chain(llm, chain_type='stuff')
    return chain.run(docs)

#### Get Bing Search API ready

In [36]:
import os
os.environ["BING_SUBSCRIPTION_KEY"] = BING_SUBSCRIPTION_KEY
os.environ["BING_SEARCH_URL"] = "https://api.bing.microsoft.com/v7.0/search"

from langchain.utilities import BingSearchAPIWrapper
search = BingSearchAPIWrapper()
# search.run("python")
html_string = search.run("When Sam Altman got fired?")
generate_response(html_string)

'Sam Altman, the co-founder and CEO of OpenAI, was fired by the company\'s board, causing shockwaves in the AI industry. Altman had taken over as CEO in 2019 and had implemented a "capped profit" arm to attract investors. Altman\'s firing followed a pattern of clashes and conflicts in his career, including being fired by his mentor at Y Combinator.'

#### Build an agent to work wiht local .csv file and answer questions based on .csv file

In [23]:
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent
llm = AzureChatOpenAI(azure_deployment="gpt-35-turbo-16k",
                      temperature=0,
                      azure_endpoint = AZURE_OPENAI_ENDPOINT,
                      openai_api_type = "azure",
                      api_key = AZURE_OPENAI_API_KEY)
agent = create_csv_agent(
    llm = llm,
    path = "./data_source/titanic.csv",
    verbose=False,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [24]:
agent.run("how many rows are there?")

'There are 891 rows in the dataframe.'

In [25]:
agent.run("how many people have more than 2 siblings")

'There are 46 people in the dataframe who have more than 2 siblings.'

In [35]:
agent.run("how many people have more than 7 siblings and who are they? Please show siblings number in a table.")

'There are 7 people who have more than 7 siblings. Here is the table showing their names and the number of siblings they have:\n\n|    | Name                               |   SibSp |\n|---:|:-----------------------------------|--------:|\n| 159 | Sage, Master. Thomas Henry         |       8 |\n| 180 | Sage, Miss. Constance Gladys       |       8 |\n| 201 | Sage, Mr. Frederick                |       8 |\n| 324 | Sage, Mr. George John Jr           |       8 |\n| 792 | Sage, Miss. Stella Anna            |       8 |\n| 846 | Sage, Mr. Douglas Bullen           |       8 |\n| 863 | Sage, Miss. Dorothy Edith "Dolly"  |       8 |'